<a href="https://colab.research.google.com/github/antonItachi/quantify-text/blob/main/upiupi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import transformers, datasets